# Statistic Analysis

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency, fisher_exact
import warnings
from typing import Union, Tuple, Dict, Any

def mcnemar_analysis_from_csv(
    model1_csv: str,
    model2_csv: str,
    model1_name: str = "Model_1",
    model2_name: str = "Model_2",
    predicted_col: str = "predicted",
    groundtruth_col: str = "groundtruth",
    alpha: float = 0.05,
    exact_threshold: int = 25,
    return_raw_data: bool = False
) -> Union[pd.DataFrame, Tuple[pd.DataFrame, Dict[str, Any]]]:
    """
    Perform comprehensive McNemar's test analysis comparing two models from CSV files.
    
    This function is designed for research paper reporting and includes all necessary
    statistical measures, effect sizes, and interpretations.
    
    Parameters:
    -----------
    model1_csv : str
        Path to CSV file with Model 1 results (columns: predicted, groundtruth)
    model2_csv : str  
        Path to CSV file with Model 2 results (columns: predicted, groundtruth)
    model1_name : str, default "Model_1"
        Name for Model 1 (for reporting)
    model2_name : str, default "Model_2"
        Name for Model 2 (for reporting)
    predicted_col : str, default "predicted"
        Name of the prediction column in CSV files
    groundtruth_col : str, default "groundtruth"
        Name of the ground truth column in CSV files
    alpha : float, default 0.05
        Significance level for statistical tests
    exact_threshold : int, default 25
        Threshold for using exact vs asymptotic test (based on discordant pairs)
    return_raw_data : bool, default False
        Whether to return raw data and intermediate calculations
        
    Returns:
    --------
    pd.DataFrame
        Comprehensive results table for research paper reporting
    Optional[Dict] (if return_raw_data=True)
        Raw data and intermediate calculations
        
    Example:
    --------
    results_df = mcnemar_analysis_from_csv(
        model1_csv="signorino_results.csv",
        model2_csv="quantum_signorino_results.csv", 
        model1_name="Signorino_Baseline",
        model2_name="Quantum_Like_Signorino"
    )
    print(results_df)
    """
    
    # Load and validate data
    try:
        df1 = pd.read_csv(model1_csv)
        df2 = pd.read_csv(model2_csv)
    except Exception as e:
        raise FileNotFoundError(f"Error reading CSV files: {e}")
    
    # Validate required columns
    required_cols = [predicted_col, groundtruth_col]
    for col in required_cols:
        if col not in df1.columns:
            raise ValueError(f"Column '{col}' not found in {model1_csv}")
        if col not in df2.columns:
            raise ValueError(f"Column '{col}' not found in {model2_csv}")
    
    # Validate same length and ground truth
    if len(df1) != len(df2):
        raise ValueError("CSV files must have the same number of observations")
    
    # Extract data
    model1_pred = df1[predicted_col].values
    model1_truth = df1[groundtruth_col].values
    model2_pred = df2[predicted_col].values  
    model2_truth = df2[groundtruth_col].values
    
    # Validate same ground truth
    if not np.array_equal(model1_truth, model2_truth):
        raise ValueError("Ground truth must be identical in both files")
    
    ground_truth = model1_truth
    n_observations = len(ground_truth)
    
    # Calculate correctness for each model
    model1_correct = (model1_pred == ground_truth).astype(int)
    model2_correct = (model2_pred == ground_truth).astype(int)
    
    # Calculate basic accuracies
    model1_accuracy = np.mean(model1_correct)
    model2_accuracy = np.mean(model2_correct)
    accuracy_diff = model2_accuracy - model1_accuracy
    
    # Create 2x2 contingency table
    a = np.sum((model1_correct == 1) & (model2_correct == 1))  # Both correct
    b = np.sum((model1_correct == 1) & (model2_correct == 0))  # Only Model 1 correct
    c = np.sum((model1_correct == 0) & (model2_correct == 1))  # Only Model 2 correct  
    d = np.sum((model1_correct == 0) & (model2_correct == 0))  # Both incorrect
    
    # Verify contingency table sums correctly
    assert a + b + c + d == n_observations, "Contingency table error"
    
    contingency_matrix = np.array([[a, b], [c, d]])
    discordant_pairs = b + c
    
    # Determine which test to use
    use_exact = discordant_pairs < exact_threshold
    
    # Perform McNemar's test
    if discordant_pairs == 0:
        # No discordant pairs - models perform identically
        mcnemar_statistic = 0.0
        p_value_asymptotic = 1.0
        p_value_exact = 1.0
        test_used = "No test needed"
        significant = False
    else:
        # Asymptotic McNemar's test (with continuity correction)
        mcnemar_statistic = (abs(b - c) - 1)**2 / discordant_pairs
        p_value_asymptotic = 1 - stats.chi2.cdf(mcnemar_statistic, df=1)
        
        # Exact McNemar's test (binomial test)
        # Use binomtest for newer scipy versions, fallback to older method
        try:
            p_value_exact = stats.binomtest(min(b, c), discordant_pairs, p=0.5, alternative='two-sided').pvalue
        except AttributeError:
            # Fallback for older scipy versions
            try:
                p_value_exact = stats.binom_test(min(b, c), discordant_pairs, p=0.5, alternative='two-sided')
            except AttributeError:
                # Manual calculation if neither function is available
                from scipy.stats import binom
                p_value_exact = 2 * min(
                    binom.cdf(min(b, c), discordant_pairs, 0.5),
                    1 - binom.cdf(min(b, c) - 1, discordant_pairs, 0.5)
                )
        
        # Choose appropriate test
        if use_exact:
            test_used = "Exact McNemar"
            p_value_reported = p_value_exact
        else:
            test_used = "Asymptotic McNemar"
            p_value_reported = p_value_asymptotic
            
        significant = p_value_reported < alpha
    
    # Calculate effect sizes and confidence intervals
    
    # Odds Ratio
    if b * c > 0:
        odds_ratio = b / c
        log_or = np.log(odds_ratio)
        se_log_or = np.sqrt(1/b + 1/c)
        or_ci_lower = np.exp(log_or - 1.96 * se_log_or)
        or_ci_upper = np.exp(log_or + 1.96 * se_log_or)
    else:
        odds_ratio = np.inf if b > c else 0 if c > b else 1
        or_ci_lower = np.nan
        or_ci_upper = np.nan
    
    # Cohen's g (effect size for McNemar's test)
    if discordant_pairs > 0:
        cohens_g = (b - c) / np.sqrt(discordant_pairs)
    else:
        cohens_g = 0
    
    # Bootstrap confidence interval for accuracy difference
    np.random.seed(42)  # For reproducibility
    n_bootstrap = 10000
    bootstrap_diffs = []
    
    for _ in range(n_bootstrap):
        # Resample with replacement
        indices = np.random.choice(n_observations, n_observations, replace=True)
        boot_acc1 = np.mean(model1_correct[indices])
        boot_acc2 = np.mean(model2_correct[indices])
        bootstrap_diffs.append(boot_acc2 - boot_acc1)
    
    acc_diff_ci_lower = np.percentile(bootstrap_diffs, 2.5)
    acc_diff_ci_upper = np.percentile(bootstrap_diffs, 97.5)
    
    # Power analysis (post-hoc)
    if discordant_pairs > 0:
        observed_p = max(b, c) / discordant_pairs  # Probability of the more frequent outcome
        # Power for detecting this effect size with current sample
        from scipy.stats import norm
        effect_size = 2 * abs(observed_p - 0.5)  # Distance from null hypothesis
        z_alpha = norm.ppf(1 - alpha/2)
        z_beta = (effect_size * np.sqrt(discordant_pairs) - z_alpha)
        power = norm.cdf(z_beta)
    else:
        power = np.nan
    
    # Create comprehensive results DataFrame
    results = {
        # Study Design
        'Comparison': f"{model1_name} vs {model2_name}",
        'Sample_Size': n_observations,
        'Significance_Level': alpha,
        
        # Model Performance
        f'{model1_name}_Accuracy': model1_accuracy,
        f'{model2_name}_Accuracy': model2_accuracy, 
        'Accuracy_Difference': accuracy_diff,
        'Accuracy_Diff_95CI_Lower': acc_diff_ci_lower,
        'Accuracy_Diff_95CI_Upper': acc_diff_ci_upper,
        
        # Contingency Table
        'Both_Correct': a,
        'Model1_Only_Correct': b,
        'Model2_Only_Correct': c,
        'Both_Incorrect': d,
        'Discordant_Pairs': discordant_pairs,
        
        # Statistical Test Results
        'Test_Used': test_used,
        'McNemar_Statistic': mcnemar_statistic,
        'P_Value_Asymptotic': p_value_asymptotic,
        'P_Value_Exact': p_value_exact,
        'P_Value_Reported': p_value_reported if discordant_pairs > 0 else p_value_exact,
        'Significant': significant,
        
        # Effect Sizes
        'Odds_Ratio': odds_ratio,
        'OR_95CI_Lower': or_ci_lower,
        'OR_95CI_Upper': or_ci_upper,
        'Cohens_g': cohens_g,
        
        # Additional Statistics
        'Power': power,
        'Model1_Wins': b,
        'Model2_Wins': c,
        'Model2_Better': c > b if discordant_pairs > 0 else False,
        
        # Interpretation Helpers
        'Effect_Size_Interpretation': _interpret_effect_size(abs(cohens_g)),
        'Statistical_Conclusion': _statistical_conclusion(significant, p_value_reported if discordant_pairs > 0 else p_value_exact, alpha),
        'Practical_Conclusion': _practical_conclusion(accuracy_diff, significant, model2_name, model1_name)
    }
    
    # Convert to DataFrame
    results_df = pd.DataFrame([results])
    
    # Format numeric columns for publication
    numeric_cols = ['Sample_Size', 'Both_Correct', 'Model1_Only_Correct', 
                   'Model2_Only_Correct', 'Both_Incorrect', 'Discordant_Pairs',
                   'Model1_Wins', 'Model2_Wins']
    
    for col in numeric_cols:
        if col in results_df.columns:
            results_df[col] = results_df[col].astype(int)
    
    # Round numeric columns appropriately
    decimal_cols = {
        'Accuracy_Difference': 4,
        'Accuracy_Diff_95CI_Lower': 4, 
        'Accuracy_Diff_95CI_Upper': 4,
        'McNemar_Statistic': 3,
        'P_Value_Asymptotic': 6,
        'P_Value_Exact': 6,
        'P_Value_Reported': 6,
        'Odds_Ratio': 3,
        'OR_95CI_Lower': 3,
        'OR_95CI_Upper': 3,
        'Cohens_g': 3,
        'Power': 3
    }
    
    for col, decimals in decimal_cols.items():
        if col in results_df.columns:
            results_df[col] = results_df[col].round(decimals)
    
    # Format accuracy columns as percentages for display
    accuracy_cols = [f'{model1_name}_Accuracy', f'{model2_name}_Accuracy']
    for col in accuracy_cols:
        if col in results_df.columns:
            results_df[col] = results_df[col].round(4)
    
    if return_raw_data:
        raw_data = {
            'model1_predictions': model1_pred,
            'model2_predictions': model2_pred,
            'ground_truth': ground_truth,
            'model1_correct': model1_correct,
            'model2_correct': model2_correct,
            'contingency_matrix': contingency_matrix,
            'bootstrap_diffs': bootstrap_diffs
        }
        return results_df, raw_data
    
    return results_df

def _interpret_effect_size(cohens_g: float) -> str:
    """Interpret Cohen's g effect size."""
    if np.isnan(cohens_g) or cohens_g == 0:
        return "No effect"
    elif abs(cohens_g) < 0.2:
        return "Small effect"
    elif abs(cohens_g) < 0.5:
        return "Medium effect" 
    else:
        return "Large effect"

def _statistical_conclusion(significant: bool, p_value: float, alpha: float) -> str:
    """Generate statistical conclusion text."""
    if significant:
        return f"Statistically significant difference (p = {p_value:.6f} < α = {alpha})"
    else:
        return f"No statistically significant difference (p = {p_value:.6f} ≥ α = {alpha})"

def _practical_conclusion(acc_diff: float, significant: bool, model2_name: str, model1_name: str) -> str:
    """Generate practical conclusion text."""
    if not significant:
        return "No evidence of systematic difference in model performance"
    
    better_model = model2_name if acc_diff > 0 else model1_name
    worse_model = model1_name if acc_diff > 0 else model2_name
    
    diff_pct = abs(acc_diff) * 100
    
    if diff_pct < 1:
        magnitude = "minimal"
    elif diff_pct < 3:
        magnitude = "small"
    elif diff_pct < 5:
        magnitude = "moderate"
    else:
        magnitude = "substantial"
    
    return f"{better_model} performs significantly better than {worse_model} ({magnitude} improvement: {diff_pct:.2f} percentage points)"

def create_publication_table(results_df: pd.DataFrame) -> pd.DataFrame:
    """
    Create a clean table formatted for research paper publication.
    """
    
    # Select key columns for publication
    pub_cols = [
        'Comparison',
        'Sample_Size',
        f'{results_df.iloc[0]["Comparison"].split(" vs ")[0]}_Accuracy',
        f'{results_df.iloc[0]["Comparison"].split(" vs ")[1]}_Accuracy', 
        'Accuracy_Difference',
        'Discordant_Pairs',
        'McNemar_Statistic', 
        'P_Value_Reported',
        'Odds_Ratio',
        'Effect_Size_Interpretation',
        'Statistical_Conclusion'
    ]
    
    # Create publication table
    pub_table = results_df[pub_cols].copy()
    
    # Rename columns for publication
    new_names = {
        'Sample_Size': 'N',
        'Accuracy_Difference': 'Δ Accuracy',
        'Discordant_Pairs': 'Discordant Pairs',
        'McNemar_Statistic': "McNemar's χ²",
        'P_Value_Reported': 'p-value',
        'Odds_Ratio': 'Odds Ratio',
        'Effect_Size_Interpretation': 'Effect Size',
        'Statistical_Conclusion': 'Result'
    }
    
    pub_table = pub_table.rename(columns=new_names)
    
    return pub_table

# Example usage and testing function
def example_usage():
    """
    Example of how to use the function with sample data.
    """
    
    # Create sample CSV files for demonstration
    np.random.seed(42)
    n_samples = 500
    
    outcomes = ['ACQ1', 'ACQ2', 'CAP1', 'CAP2', 'SQ', 'NEGO', 'WAR1', 'WAR2']
    ground_truth = np.random.choice(outcomes, n_samples)
    
    # Model 1: 75% accuracy
    model1_correct = np.random.binomial(1, 0.75, n_samples)
    model1_pred = ground_truth.copy()
    error_indices = np.where(model1_correct == 0)[0]
    model1_pred[error_indices] = np.random.choice(outcomes, len(error_indices))
    
    # Model 2: 78% accuracy  
    model2_correct = np.random.binomial(1, 0.78, n_samples)
    model2_pred = ground_truth.copy()
    error_indices = np.where(model2_correct == 0)[0]
    model2_pred[error_indices] = np.random.choice(outcomes, len(error_indices))
    
    # Save sample CSV files
    pd.DataFrame({
        'predicted': model1_pred,
        'groundtruth': ground_truth
    }).to_csv('model1_results.csv', index=False)
    
    pd.DataFrame({
        'predicted': model2_pred, 
        'groundtruth': ground_truth
    }).to_csv('model2_results.csv', index=False)
    
    # Run analysis
    results = mcnemar_analysis_from_csv(
        'model1_results.csv',
        'model2_results.csv',
        model1_name='Signorino_Baseline',
        model2_name='Quantum_Like_Signorino'
    )
    
    print("Complete Statistical Analysis Results:")
    print("=" * 50)
    print(results.T)  # Transpose for better readability
    
    print("\n\nPublication-Ready Table:")
    print("=" * 30)
    pub_table = create_publication_table(results)
    print(pub_table.to_string(index=False))
    
    return results

if __name__ == "__main__":
    # Run example
    example_results = example_usage()

Complete Statistical Analysis Results:
                                                                                 0
Comparison                            Signorino_Baseline vs Quantum_Like_Signorino
Sample_Size                                                                    500
Significance_Level                                                            0.05
Signorino_Baseline_Accuracy                                                  0.788
Quantum_Like_Signorino_Accuracy                                              0.782
Accuracy_Difference                                                         -0.006
Accuracy_Diff_95CI_Lower                                                    -0.056
Accuracy_Diff_95CI_Upper                                                     0.044
Both_Correct                                                                   311
Model1_Only_Correct                                                             83
Model2_Only_Correct                             

In [4]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency, fisher_exact
import warnings
from typing import Union, Tuple, Dict, Any

def exact_mcnemar_analysis_from_csv(
    model1_csv: str,
    model2_csv: str,
    model1_name: str = "Model_1",
    model2_name: str = "Model_2",
    predicted_col: str = "predicted",
    groundtruth_col: str = "groundtruth",
    alpha: float = 0.05,
    return_raw_data: bool = False
) -> Union[pd.DataFrame, Tuple[pd.DataFrame, Dict[str, Any]]]:
    """
    Perform comprehensive EXACT McNemar's test analysis comparing two models from CSV files.
    
    This function uses ONLY the exact binomial method for McNemar's test, regardless of sample size.
    Designed for research paper reporting with all necessary statistical measures.
    
    Parameters:
    -----------
    model1_csv : str
        Path to CSV file with Model 1 results (columns: predicted, groundtruth)
    model2_csv : str  
        Path to CSV file with Model 2 results (columns: predicted, groundtruth)
    model1_name : str, default "Model_1"
        Name for Model 1 (for reporting)
    model2_name : str, default "Model_2"
        Name for Model 2 (for reporting)
    predicted_col : str, default "predicted"
        Name of the prediction column in CSV files
    groundtruth_col : str, default "groundtruth"
        Name of the ground truth column in CSV files
    alpha : float, default 0.05
        Significance level for statistical tests
    return_raw_data : bool, default False
        Whether to return raw data and intermediate calculations
        
    Returns:
    --------
    pd.DataFrame
        Comprehensive results table for research paper reporting (EXACT method only)
    Optional[Dict] (if return_raw_data=True)
        Raw data and intermediate calculations
        
    Example:
    --------
    results_df = exact_mcnemar_analysis_from_csv(
        model1_csv="signorino_results.csv",
        model2_csv="quantum_signorino_results.csv", 
        model1_name="Signorino_Baseline",
        model2_name="Quantum_Like_Signorino"
    )
    print(results_df)
    """
    
    # Load and validate data
    try:
        df1 = pd.read_csv(model1_csv)
        df2 = pd.read_csv(model2_csv)
    except Exception as e:
        raise FileNotFoundError(f"Error reading CSV files: {e}")
    
    # Validate required columns
    required_cols = [predicted_col, groundtruth_col]
    for col in required_cols:
        if col not in df1.columns:
            raise ValueError(f"Column '{col}' not found in {model1_csv}")
        if col not in df2.columns:
            raise ValueError(f"Column '{col}' not found in {model2_csv}")
    
    # Validate same length and ground truth
    if len(df1) != len(df2):
        raise ValueError("CSV files must have the same number of observations")
    
    # Extract data
    model1_pred = df1[predicted_col].values
    model1_truth = df1[groundtruth_col].values
    model2_pred = df2[predicted_col].values  
    model2_truth = df2[groundtruth_col].values
    
    # Validate same ground truth
    if not np.array_equal(model1_truth, model2_truth):
        raise ValueError("Ground truth must be identical in both files")
    
    ground_truth = model1_truth
    n_observations = len(ground_truth)
    
    # Calculate correctness for each model
    model1_correct = (model1_pred == ground_truth).astype(int)
    model2_correct = (model2_pred == ground_truth).astype(int)
    
    # Calculate basic accuracies
    model1_accuracy = np.mean(model1_correct)
    model2_accuracy = np.mean(model2_correct)
    accuracy_diff = model2_accuracy - model1_accuracy
    
    # Create 2x2 contingency table
    a = np.sum((model1_correct == 1) & (model2_correct == 1))  # Both correct
    b = np.sum((model1_correct == 1) & (model2_correct == 0))  # Only Model 1 correct
    c = np.sum((model1_correct == 0) & (model2_correct == 1))  # Only Model 2 correct  
    d = np.sum((model1_correct == 0) & (model2_correct == 0))  # Both incorrect
    
    # Verify contingency table sums correctly
    assert a + b + c + d == n_observations, "Contingency table error"
    
    contingency_matrix = np.array([[a, b], [c, d]])
    discordant_pairs = b + c
    
    # Perform EXACT McNemar's test
    if discordant_pairs == 0:
        # No discordant pairs - models perform identically
        p_value_exact = 1.0
        test_used = "No test needed (identical performance)"
        significant = False
        binomial_statistic = 0
        expected_wins_model1 = 0
        expected_wins_model2 = 0
    else:
        # EXACT McNemar's test using binomial distribution
        test_used = "Exact McNemar's Test (Binomial)"
        
        # Use binomtest for newer scipy versions, fallback to older method
        try:
            # For exact test, we test if min(b,c) follows Binomial(b+c, 0.5)
            binomial_result = stats.binomtest(min(b, c), discordant_pairs, p=0.5, alternative='two-sided')
            p_value_exact = binomial_result.pvalue
        except AttributeError:
            # Fallback for older scipy versions
            try:
                p_value_exact = stats.binom_test(min(b, c), discordant_pairs, p=0.5, alternative='two-sided')
            except AttributeError:
                # Manual calculation if neither function is available
                from scipy.stats import binom
                p_value_exact = 2 * min(
                    binom.cdf(min(b, c), discordant_pairs, 0.5),
                    1 - binom.cdf(min(b, c) - 1, discordant_pairs, 0.5)
                )
        
        # Binomial test statistic (number of successes)
        binomial_statistic = min(b, c)
        
        # Expected values under null hypothesis
        expected_wins_model1 = discordant_pairs / 2
        expected_wins_model2 = discordant_pairs / 2
        
        significant = p_value_exact < alpha
    
    # Calculate exact confidence interval for the probability
    if discordant_pairs > 0:
        # Exact confidence interval for P(Model 1 wins | models disagree)
        p_model1_wins = b / discordant_pairs
        
        # Use beta distribution for exact binomial confidence interval
        from scipy.stats import beta
        alpha_level = 1 - 0.95  # for 95% CI
        
        if b == 0:
            ci_lower = 0
            ci_upper = 1 - (alpha_level/2)**(1/discordant_pairs)
        elif b == discordant_pairs:
            ci_lower = (alpha_level/2)**(1/discordant_pairs)
            ci_upper = 1
        else:
            ci_lower = beta.ppf(alpha_level/2, b, discordant_pairs - b + 1)
            ci_upper = beta.ppf(1 - alpha_level/2, b + 1, discordant_pairs - b)
    else:
        p_model1_wins = 0.5
        ci_lower = 0.0
        ci_upper = 1.0
    
    # Calculate effect sizes
    
    # Exact Odds Ratio with exact confidence interval
    if b * c > 0:
        odds_ratio = b / c
        
        # Exact confidence interval for odds ratio using Fisher's method
        # This is more accurate than the log-normal approximation
        try:
            _, fisher_p_value = fisher_exact([[a, b], [c, d]], alternative='two-sided')
            
            # For exact OR confidence interval, use conditional maximum likelihood
            # This is computationally intensive, so we'll use approximation but note it's exact test
            log_or = np.log(odds_ratio)
            se_log_or = np.sqrt(1/b + 1/c)
            or_ci_lower = np.exp(log_or - 1.96 * se_log_or)
            or_ci_upper = np.exp(log_or + 1.96 * se_log_or)
        except:
            or_ci_lower = np.nan
            or_ci_upper = np.nan
            fisher_p_value = p_value_exact
    else:
        odds_ratio = np.inf if b > c else 0 if c > b else 1
        or_ci_lower = np.nan
        or_ci_upper = np.nan
        fisher_p_value = p_value_exact
    
    # Cohen's g (effect size for McNemar's test)
    if discordant_pairs > 0:
        cohens_g = (b - c) / np.sqrt(discordant_pairs)
    else:
        cohens_g = 0
    
    # Exact bootstrap confidence interval for accuracy difference
    np.random.seed(42)  # For reproducibility
    n_bootstrap = 10000
    bootstrap_diffs = []
    
    for _ in range(n_bootstrap):
        # Resample with replacement
        indices = np.random.choice(n_observations, n_observations, replace=True)
        boot_acc1 = np.mean(model1_correct[indices])
        boot_acc2 = np.mean(model2_correct[indices])
        bootstrap_diffs.append(boot_acc2 - boot_acc1)
    
    acc_diff_ci_lower = np.percentile(bootstrap_diffs, 2.5)
    acc_diff_ci_upper = np.percentile(bootstrap_diffs, 97.5)
    
    # Power analysis for exact test
    if discordant_pairs > 0:
        observed_p = max(b, c) / discordant_pairs
        effect_size = abs(observed_p - 0.5)
        
        # For exact binomial test, power calculation is more complex
        # Using normal approximation for power calculation
        from scipy.stats import norm
        z_alpha = norm.ppf(1 - alpha/2)
        z_beta = (effect_size * np.sqrt(discordant_pairs) - z_alpha)
        power = norm.cdf(z_beta)
    else:
        power = np.nan
        effect_size = 0
    
    # Additional exact statistics
    if discordant_pairs > 0:
        # Exact probability of observing this result or more extreme under H0
        from scipy.stats import binom
        prob_current_or_more_extreme = 1 - binom.cdf(max(b, c) - 1, discordant_pairs, 0.5)
        prob_current_or_more_extreme *= 2  # Two-sided
        
        # Minimum detectable difference with current sample size
        min_detectable_diff = 1.96 * np.sqrt(0.25 / discordant_pairs)
    else:
        prob_current_or_more_extreme = 1.0
        min_detectable_diff = np.nan
    
    # Create comprehensive results DataFrame
    results = {
        # Study Design
        'Comparison': f"{model1_name} vs {model2_name}",
        'Sample_Size': n_observations,
        'Significance_Level': alpha,
        'Test_Method': "Exact McNemar's Test",
        
        # Model Performance
        f'{model1_name}_Accuracy': model1_accuracy,
        f'{model2_name}_Accuracy': model2_accuracy, 
        'Accuracy_Difference': accuracy_diff,
        'Accuracy_Diff_95CI_Lower': acc_diff_ci_lower,
        'Accuracy_Diff_95CI_Upper': acc_diff_ci_upper,
        
        # Contingency Table
        'Both_Correct': a,
        'Model1_Only_Correct': b,
        'Model2_Only_Correct': c,
        'Both_Incorrect': d,
        'Discordant_Pairs': discordant_pairs,
        
        # Exact Test Results
        'Test_Used': test_used,
        'Binomial_Statistic': binomial_statistic,
        'Expected_Model1_Wins': expected_wins_model1,
        'Expected_Model2_Wins': expected_wins_model2,
        'P_Value_Exact': p_value_exact,
        'Fisher_Exact_P_Value': fisher_p_value if discordant_pairs > 0 else p_value_exact,
        'Significant': significant,
        
        # Exact Probability Estimates
        'P_Model1_Wins_Given_Disagreement': p_model1_wins,
        'P_Model1_Wins_CI_Lower': ci_lower,
        'P_Model1_Wins_CI_Upper': ci_upper,
        
        # Effect Sizes
        'Odds_Ratio': odds_ratio,
        'OR_95CI_Lower': or_ci_lower,
        'OR_95CI_Upper': or_ci_upper,
        'Cohens_g': cohens_g,
        
        # Additional Exact Statistics
        'Power': power,
        'Effect_Size': effect_size,
        'Prob_Current_Or_More_Extreme': prob_current_or_more_extreme,
        'Min_Detectable_Difference': min_detectable_diff,
        'Model1_Wins': b,
        'Model2_Wins': c,
        'Model2_Better': c > b if discordant_pairs > 0 else False,
        
        # Interpretation Helpers
        'Effect_Size_Interpretation': _interpret_effect_size(abs(cohens_g)),
        'Statistical_Conclusion': _statistical_conclusion(significant, p_value_exact, alpha),
        'Practical_Conclusion': _practical_conclusion(accuracy_diff, significant, model2_name, model1_name),
        'Exact_Test_Interpretation': _exact_test_interpretation(b, c, discordant_pairs, p_value_exact, alpha)
    }
    
    # Convert to DataFrame
    results_df = pd.DataFrame([results])
    
    # Format numeric columns for publication
    numeric_cols = ['Sample_Size', 'Both_Correct', 'Model1_Only_Correct', 
                   'Model2_Only_Correct', 'Both_Incorrect', 'Discordant_Pairs',
                   'Model1_Wins', 'Model2_Wins', 'Binomial_Statistic']
    
    for col in numeric_cols:
        if col in results_df.columns:
            results_df[col] = results_df[col].astype(int)
    
    # Round numeric columns appropriately
    decimal_cols = {
        'Accuracy_Difference': 4,
        'Accuracy_Diff_95CI_Lower': 4, 
        'Accuracy_Diff_95CI_Upper': 4,
        'Expected_Model1_Wins': 1,
        'Expected_Model2_Wins': 1,
        'P_Value_Exact': 6,
        'Fisher_Exact_P_Value': 6,
        'P_Model1_Wins_Given_Disagreement': 4,
        'P_Model1_Wins_CI_Lower': 4,
        'P_Model1_Wins_CI_Upper': 4,
        'Odds_Ratio': 3,
        'OR_95CI_Lower': 3,
        'OR_95CI_Upper': 3,
        'Cohens_g': 3,
        'Power': 3,
        'Effect_Size': 4,
        'Prob_Current_Or_More_Extreme': 6,
        'Min_Detectable_Difference': 4
    }
    
    for col, decimals in decimal_cols.items():
        if col in results_df.columns:
            results_df[col] = results_df[col].round(decimals)
    
    # Format accuracy columns
    accuracy_cols = [f'{model1_name}_Accuracy', f'{model2_name}_Accuracy']
    for col in accuracy_cols:
        if col in results_df.columns:
            results_df[col] = results_df[col].round(4)
    
    if return_raw_data:
        raw_data = {
            'model1_predictions': model1_pred,
            'model2_predictions': model2_pred,
            'ground_truth': ground_truth,
            'model1_correct': model1_correct,
            'model2_correct': model2_correct,
            'contingency_matrix': contingency_matrix,
            'bootstrap_diffs': bootstrap_diffs,
            'discordant_pairs_breakdown': {'model1_wins': b, 'model2_wins': c}
        }
        return results_df, raw_data
    
    return results_df

def _interpret_effect_size(cohens_g: float) -> str:
    """Interpret Cohen's g effect size."""
    if np.isnan(cohens_g) or cohens_g == 0:
        return "No effect"
    elif abs(cohens_g) < 0.2:
        return "Small effect"
    elif abs(cohens_g) < 0.5:
        return "Medium effect" 
    else:
        return "Large effect"

def _statistical_conclusion(significant: bool, p_value: float, alpha: float) -> str:
    """Generate statistical conclusion text."""
    if significant:
        return f"Statistically significant difference (exact p = {p_value:.6f} < α = {alpha})"
    else:
        return f"No statistically significant difference (exact p = {p_value:.6f} ≥ α = {alpha})"

def _practical_conclusion(acc_diff: float, significant: bool, model2_name: str, model1_name: str) -> str:
    """Generate practical conclusion text."""
    if not significant:
        return "No evidence of systematic difference in model performance"
    
    better_model = model2_name if acc_diff > 0 else model1_name
    worse_model = model1_name if acc_diff > 0 else model2_name
    
    diff_pct = abs(acc_diff) * 100
    
    if diff_pct < 1:
        magnitude = "minimal"
    elif diff_pct < 3:
        magnitude = "small"
    elif diff_pct < 5:
        magnitude = "moderate"
    else:
        magnitude = "substantial"
    
    return f"{better_model} performs significantly better than {worse_model} ({magnitude} improvement: {diff_pct:.2f} percentage points)"

def _exact_test_interpretation(b: int, c: int, discordant_pairs: int, p_value: float, alpha: float) -> str:
    """Generate interpretation specific to exact test."""
    if discordant_pairs == 0:
        return "Models perform identically - no statistical test needed"
    
    total_disagreements = discordant_pairs
    model1_wins = b
    model2_wins = c
    
    interpretation = f"Of {total_disagreements} cases where models disagreed: "
    interpretation += f"Model 1 was correct in {model1_wins} cases, Model 2 in {model2_wins} cases. "
    
    if p_value < alpha:
        winner = "Model 1" if model1_wins > model2_wins else "Model 2"
        interpretation += f"This difference is statistically significant (exact p = {p_value:.6f}), "
        interpretation += f"indicating {winner} systematically outperforms when models disagree."
    else:
        interpretation += f"This difference is not statistically significant (exact p = {p_value:.6f}), "
        interpretation += "suggesting no systematic advantage for either model."
    
    return interpretation

def create_exact_publication_table(results_df: pd.DataFrame) -> pd.DataFrame:
    """
    Create a clean table formatted for research paper publication (exact method).
    """
    
    comparison_parts = results_df.iloc[0]["Comparison"].split(" vs ")
    model1_acc_col = f'{comparison_parts[0]}_Accuracy'
    model2_acc_col = f'{comparison_parts[1]}_Accuracy'
    
    # Select key columns for publication
    pub_cols = [
        'Comparison',
        'Sample_Size',
        model1_acc_col,
        model2_acc_col, 
        'Accuracy_Difference',
        'Discordant_Pairs',
        'P_Value_Exact',
        'P_Model1_Wins_Given_Disagreement',
        'Odds_Ratio',
        'Effect_Size_Interpretation',
        'Exact_Test_Interpretation'
    ]
    
    # Create publication table
    pub_table = results_df[pub_cols].copy()
    
    # Rename columns for publication
    new_names = {
        'Sample_Size': 'N',
        'Accuracy_Difference': 'Δ Accuracy',
        'Discordant_Pairs': 'Disagreements',
        'P_Value_Exact': 'Exact p-value',
        'P_Model1_Wins_Given_Disagreement': 'P(Model1 wins | disagree)',
        'Odds_Ratio': 'Odds Ratio',
        'Effect_Size_Interpretation': 'Effect Size',
        'Exact_Test_Interpretation': 'Result'
    }
    
    pub_table = pub_table.rename(columns=new_names)
    
    return pub_table

# Example usage function
def example_exact_usage():
    """
    Example of how to use the exact McNemar's test function.
    """
    
    # Create sample CSV files for demonstration
    np.random.seed(42)
   # n_samples = 100  # Smaller sample to show exact vs asymptotic differences
   # 
   # outcomes = ['ACQ1', 'ACQ2', 'CAP1', 'CAP2', 'SQ', 'NEGO', 'WAR1', 'WAR2']
   # ground_truth = np.random.choice(outcomes, n_samples)
    
    # Model 1: 75% accuracy
   # model1_correct = np.random.binomial(1, 0.75, n_samples)
   # model1_pred = ground_truth.copy()
   # error_indices = np.where(model1_correct == 0)[0]
   # model1_pred[error_indices] = np.random.choice(outcomes, len(error_indices))
    
    # Model 2: 80% accuracy  
   # model2_correct = np.random.binomial(1, 0.80, n_samples)
   # model2_pred = ground_truth.copy()
   # error_indices = np.where(model2_correct == 0)[0]
   # model2_pred[error_indices] = np.random.choice(outcomes, len(error_indices))
    
    # Save sample CSV files
   # pd.DataFrame({
   #     'predicted': model1_pred,
   #     'groundtruth': ground_truth
   # }).to_csv('exact_model1_results.csv', index=False)
    
   # pd.DataFrame({
   #     'predicted': model2_pred, 
   #     'groundtruth': ground_truth
   # }).to_csv('exact_model2_results.csv', index=False)
    
    # Run EXACT analysis
    results = exact_mcnemar_analysis_from_csv(
        'model1_results.csv',
        'model2_results.csv',
        model1_name='Signorino_Baseline',
        model2_name='Quantum_Like_Signorino'
    )
    
    print("EXACT McNemar's Test Analysis Results:")
    print("=" * 55)
    print(results.T)  # Transpose for better readability
    
    print("\n\nExact Test Publication-Ready Table:")
    print("=" * 40)
    pub_table = create_exact_publication_table(results)
    print(pub_table.to_string(index=False))
    
    return results

if __name__ == "__main__":
    # Run example
    exact_results = example_exact_usage()

EXACT McNemar's Test Analysis Results:
                                                                                  0
Comparison                             Signorino_Baseline vs Quantum_Like_Signorino
Sample_Size                                                                     500
Significance_Level                                                             0.05
Test_Method                                                    Exact McNemar's Test
Signorino_Baseline_Accuracy                                                   0.788
Quantum_Like_Signorino_Accuracy                                               0.782
Accuracy_Difference                                                          -0.006
Accuracy_Diff_95CI_Lower                                                     -0.056
Accuracy_Diff_95CI_Upper                                                      0.044
Both_Correct                                                                    311
Model1_Only_Correct                  